In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as sts
from math import *
import glob
from IPython.display import display
# from scipy.linalg import block_diag

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Сливаем таблицы в один дата фрейм
names = glob.glob(r'/content/drive/MyDrive/Андан Проект/Данные индексов/*.csv',\
                  recursive=True)
ticker_names = [x.replace('.csv', '').split()[-1] for x in names]
merged = pd.read_csv(names[0])
merged = merged.rename(str.lower, axis=1).rename(columns={' ime': 'time',\
                                                 'open': f'open_{ticker_names[0]}',\
                                                 'high': f'high_{ticker_names[0]}',\
                                                 'low': f'low_{ticker_names[0]}',\
                                                 'close': f'close_{ticker_names[0]}',\
                                                 'volume': f'volume_{ticker_names[0]}',\
                                                 })
for i in range(1, len(names)):
    df = pd.read_csv(names[i])
    df = df.rename(str.lower, axis=1).rename(columns={' ime': 'time',\
                                                 'open': f'open_{ticker_names[i]}',\
                                                 'high': f'high_{ticker_names[i]}',\
                                                 'low': f'low_{ticker_names[i]}',\
                                                 'close': f'close_{ticker_names[i]}',\
                                                 'volume': f'volume_{ticker_names[i]}',\
                                                 })
    merged = merged.merge(df, on='time')
    df_indexes = merged

In [ ]:
df_indexes['time'] = pd.to_datetime(df_indexes['time'])

In [ ]:
# Добавляем новый признак логарифмическая доходность за минуту
for ticker_name in ticker_names:
    df_indexes[f'log_return_{ticker_name}'] = np.log(df_indexes[f'open_{ticker_name}'].shift(1) / \
    df_indexes[f'open_{ticker_name}'])

In [ ]:
# Провальная попытка сделать матрично
# n, d = 390, 13
# blocks = [np.ones((n, n)) * 1/n] * d
# day_mean_matrix = block_diag(*blocks)[1:, :]
# for ticker_name in ticker_names:
#     np_mean_day_log_return = day_mean_matrix @ df_indexes[f'log_return_{ticker_name}'].to_numpy()
#     print(np_mean_day_log_return)
#     df_indexes[f'mean_day_log_return_{ticker_name}'] = pd.DataFrame(np_mean_day_log_return)

In [ ]:
df_indexes['open_QQQ'].corr(df_indexes['open_SPX'])

0.9831245751946432

In [ ]:
df_indexes['open_QQQ'].corr(df_indexes['open_SPY'])

0.985977407698796

In [10]:
df_indexes['open_SPX'].corr(df_indexes['open_SPY'])

0.9995977119568936

In [ ]:
# Добавляем новый признак минутная волатильность
df_indexes['date'] = df_indexes['time'].apply(lambda x: x.floor('d'))
for ticker_name in ticker_names:
    df_std = df_indexes.groupby(pd.Grouper(key='time', freq='D'))\
     [f'log_return_{ticker_name}'].std()
    df_std = df_std.reset_index()
    df_std.rename(columns={'time': 'date', f'log_return_{ticker_name}':\
                           f'min_std_log_return_{ticker_name}'}, inplace=True)
    df_indexes = df_indexes.merge(df_std, on='date', how='left')
df_indexes.drop(columns='date', inplace=True)